In [2]:
import pandas as pd
import pyodbc
import useful_functions as use
from functools import reduce

In [3]:
npi_phones = pd.read_csv('../../Data/NPI/NPI_PHONES.csv')
npi_phones['ME'] = use.fix_me(npi_phones.ME)
npi_phones['IQVIA_ME'] = [x[0:10] for x in npi_phones.ME]

In [4]:
len(npi_phones)

1069582

In [8]:
ppd = use.get_ppd()

C:/Users/vigrose/Data/PPD\ppd_data_20210703.csv


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,16,20,21,42,49,56,57,65,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
NO_CONTACT = ppd[ppd.NO_CONTACT_IND=='N'].ME

In [16]:
no_contact = use.fix_me(NO_CONTACT)

In [31]:
npi_phones[(npi_phones.TOP_CD==20)&(npi_phones.ME.isin(no_contact)==False)&(npi_phones.PHONE_PPD=='None')].sample(14000).groupby('PHONE_NPI').count().sort_values('NPI')

,NPI,ME,TOP_CD,PHONE_PPD,IQVIA_ME
PHONE_NPI,,,,,
9999999999,1,1,1,1,1
7012345997,1,1,1,1,1
7012348830,1,1,1,1,1
7012351924,1,1,1,1,1
7012544531,1,1,1,1,1
...,...,...,...,...,...
9049532000,16,16,16,16,16
7207771234,18,18,18,18,18
8437921414,23,23,23,23,23


In [3]:
#dhc
dhc = pd.read_csv('../../Data/DHC/DHC_2021-04.csv')
dhc = dhc.fillna('None')
dhc['ME'] = use.fix_me(dhc.ME)
dhc['DHC_PHONE'] = [use.fix_phone(x) for x in dhc['Phone Number']]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#data.gov
gov = pd.read_csv('../../Data/DataGov/me_gov.csv')
gov = gov.fillna('None')
gov['ME'] = use.fix_me(gov.ME)
gov['GOV_PHONE'] = [use.fix_phone(x) for x in gov[' phn_numbr']]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (14,15,25,29,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#iqvia
username = 'vigrose'
password = 'Ravenclaw~10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT DISTINCT 
        B.PHONE,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,PHONE,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,2519493400,0010200001,Admitting,A,6
1,7275885200,0010200001,Admitting,A,6
2,8282131111,0010200003,Admitting,A,6
3,6235244000,0010200004,Attending,P,10
4,5048423000,0010200005,IDN Affiliated (Inferred),P,9


In [7]:
iqvia =iqvia.fillna('None')
iqvia['IQVIA_PHONE'] = [use.fix_phone(x) for x in iqvia['PHONE']]

In [8]:
#symphony
sym_query = \
        """
        SELECT
        d.ADDR_FRST_TLPHN_NBR AS TELEPHONE,
        l.OTHER_ID AS SYM_ME
        FROM
        ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT d, ODS.PRACTITIONER_ADDL_IDS_LAYOUT l
        WHERE
        d.DS_PRCTR_ID = l.DS_PRCTR_ID
        and
        l.ID_QLFR_TYP_CDE = 38
        """  
symphony = pd.read_sql(con=ODS, sql=sym_query)
symphony.head()

,TELEPHONE,SYM_ME
0,(608) 263-6421,0540475117
1,(501) 376-1160,0540475123
2,(617) 499-5140,5750218010
3,(509) 747-4455,0540475135
4,(360) 385-7978,0540476004


In [9]:
symphony =symphony.fillna('None')
symphony['SYM_PHONE'] = [use.fix_phone(x) for x in symphony['TELEPHONE']]

In [12]:
test = ''
test+='Symphony'
test+=', DHC, '
test[:-2]

'Symphony, DHC'

In [14]:
THIS = pd.merge(npi_phones, symphony, left_on='IQVIA_ME', right_on='SYM_ME', how='left')
THIS = pd.merge(THIS, iqvia, left_on='IQVIA_ME', right_on='ME', how='left', suffixes = ['','_iqvia'])
THIS = pd.merge(THIS, dhc, on='ME', how='left')
THIS = pd.merge(THIS, gov, on='ME', how='left')[['ME','IQVIA_ME', 'PHONE_NPI', 'PHONE_PPD','SYM_PHONE','IQVIA_PHONE', 'DHC_PHONE', 'GOV_PHONE']].drop_duplicates()
THIS = THIS.fillna('None')

In [42]:
DPC = npi_phones[npi_phones.TOP_CD==20]

In [58]:
dhc = dhc[dhc.DHC_PHONE!='None']

In [63]:
gov = gov[gov.GOV_PHONE!='None']

In [45]:
symphony = symphony[symphony.SYM_PHONE!='None']

In [53]:
iqvia

,PHONE,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK,IQVIA_PHONE
0,2519493400,0010200001,Admitting,A,6,2519493400
1,7275885200,0010200001,Admitting,A,6,7275885200
2,8282131111,0010200003,Admitting,A,6,8282131111
3,6235244000,0010200004,Attending,P,10,6235244000
4,5048423000,0010200005,IDN Affiliated (Inferred),P,9,5048423000
...,...,...,...,...,...,...
1520275,9898943000,None,Attending,P,10,9898943000
1520276,9899078000,None,Admitting,A,6,9899078000
1520277,9899078000,None,Attending,A,8,9899078000
1520278,9899078000,None,Admitting,P,10,9899078000


In [65]:
pd.merge(DPC, gov, on='ME').drop_duplicates('ME')

,NPI_x,ME,TOP_CD,PHONE_PPD,PHONE_NPI,IQVIA_ME,NPI_y,Ind_PAC_ID,Ind_enrl_ID,lst_nm,...,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id,PARTY_ID,GOV_PHONE
0,1588667638,01201861256,20,9042640088,9043881820,0120186125,1588667638,4183796493,I20080702000080,PILCHER,...,ASCENSION ST VINCENT'S CLAY COUNTY,100307,ASCENSION ST VINCENT'S SOUTHSIDE,None,None,Y,Y,FL322044311JA2345XSTXX300,1878763,9043819994
9,1932102084,49558830041,20,4198423000,4198423000,4955883004,1932102084,8426956210,I20091120000045,ADUSUMILLI,...,PROMEDICA TOLEDO HOSPITAL,360090,ST LUKE'S HOSPITAL,360029,WOOD COUNTY HOSPITAL,Y,Y,MI492211465AD770XXAVEX313,2650579,5172660525
15,1750384806,04815940089,20,None,8067610540,0481594008,1750384806,6204977663,I20100107000094,BISBEE,...,None,None,None,None,None,Y,Y,TX794309406LU3601XSTXX300,2440888,8067437337
16,1578566626,74808810054,20,5208363446,6023446600,7480881005,1578566626,7810998820,I20070117000526,KUIPERS,...,None,None,None,None,None,Y,Y,AZ852832664GU5825XGUAD400,2677045,4803446000
18,1487657532,03901961441,20,9183426200,4057932229,0390196144,1487657532,4688767486,I20181207000330,HUEBERT,...,None,None,None,None,None,Y,Y,OK740175047CL101XXAVEX400,2306306,9183426200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163301,1538709985,01875840335,20,None,6416843173,0187584033,1538709985,7315845575,I20031224000045,DIELEMAN,...,None,None,None,None,None,Y,Y,IA525012186OT1001XAVEX300,2131869,6419842341
1163302,1275154452,01902170509,20,None,3528679601,0190217050,1275154452,1456772847,I20200611001568,HUGHES,...,None,None,None,None,None,Y,Y,FL344715426OC1805XAVEX501,6644028,3528679601
1163303,1912517350,06509120046,20,8008413523,2087433523,0650912004,1912517350,5496173296,I20201014003914,NETTING,...,None,None,None,None,None,Y,Y,WA994032829CL1221XAVEX300,15503761,2087461383
1163305,1922611896,00512792090,20,7149996688,7607735994,0051279209,1922611896,4284046210,I20201216002648,JOHNSON,...,None,None,None,None,None,Y,None,CA928055854AN1020XBLVD404,1812130,


In [67]:
pd.merge(npi_phones, gov, left_on=['ME','PHONE_NPI'], right_on=['ME','GOV_PHONE']).drop_duplicates('ME')

,NPI_x,ME,TOP_CD,PHONE_PPD,PHONE_NPI,IQVIA_ME,NPI_y,Ind_PAC_ID,Ind_enrl_ID,lst_nm,...,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id,PARTY_ID,GOV_PHONE
0,1588667638,01201861256,20,9042640088,9043881820,0120186125,1588667638,4183796493,I20080702000080,PILCHER,...,ASCENSION ST VINCENT'S CLAY COUNTY,100307,ASCENSION ST VINCENT'S SOUTHSIDE,None,None,Y,Y,FL322044736JA1824XSTXX301,1878763,9043881820
1,1932102084,49558830041,20,4198423000,4198423000,4955883004,1932102084,8426956210,I20031231000003,ADUSUMILLI,...,PROMEDICA TOLEDO HOSPITAL,360090,ST LUKE'S HOSPITAL,360029,WOOD COUNTY HOSPITAL,Y,Y,OH436151753TO2940XRDXX400,2650579,4198423000
2,1750384707,02501791167,20,9528318742,9528318742,0250179116,1750384707,7315914041,I20050221000142,MARKMAN,...,None,None,None,None,None,Y,Y,MN554314800BL8100XDRXX300,2141270,9528318742
3,1073516027,03306970614,20,8457355666,8457355666,0330697061,1073516027,2860436995,I20050620000184,CHANG,...,None,None,None,None,None,Y,Y,NY109652057PE169XXRDXX400,2184778,8457355666
4,1508869553,04177910339,20,8566297436,8566297436,0417791033,1508869553,1052437480,I20110304000348,VAN HOUTEN-SAUTER,...,None,None,None,None,None,Y,Y,NJ080941137WI220XXSTXX300,2570793,8566297436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226014,1922635705,05104900064,71,3042589433,7176894243,0510490006,1922635705,2567891641,I20200413000684,BAER,...,None,None,None,None,None,Y,Y,PA170222573EL99XXXDRXX302,2589310,7176894243
226015,1275154452,01902170509,20,None,3528679601,0190217050,1275154452,1456772847,I20200611001568,HUGHES,...,None,None,None,None,None,Y,Y,FL344715426OC1805XAVEX501,6644028,3528679601
226016,1255953683,02401781385,30,9014482214,9017469438,0240178138,1255953683,840365821,I20200624003364,STERN,...,None,None,None,None,None,Y,Y,TN381342926BA6630XCIRX401,2071792,9017469438
226017,1912517350,06509120046,20,8008413523,2087433523,0650912004,1912517350,5496173296,I20200910002021,NETTING,...,None,None,None,None,None,Y,Y,ID835014441LE320XXDRXX300,15503761,2087433523


In [37]:
dict_list = []
for row in THIS.itertuples():
    count = 0
    MATCHES = ''
    phone = row.PHONE_NPI
    symph_phone = row.SYM_PHONE
    dhc_phone = row.DHC_PHONE
    iqvia_phone = row.IQVIA_PHONE
    data_phone = row.GOV_PHONE
    if symph_phone == phone:
        count += 1
        MATCHES += 'Symphony, '
    if dhc_phone == phone:
        count += 1
        MATCHES += 'DHC, '
    if iqvia_phone == phone:
        count += 1
        MATCHES += 'IQVia, '
    if data_phone == phone:
        count += 1
        MATCHES += 'DataGov, '
    if phone == row.PHONE_PPD:
        count += 1
        MATCHES += 'Masterfile, '
    if phone!='None':
        dicto = {
        'ME': row.ME,
        'IQVIA_ME': row.IQVIA_ME,
        'PHONE_NPI': phone,
        'PHONE_PPD': row.PHONE_PPD,
        'MATCHED': count,
        'MATCHES': MATCHES[:-2]
    }
        dict_list.append(dicto)

In [33]:
phone_matches = pd.DataFrame(dict_list)

In [35]:
phone_matches = phone_matches.sort_values('MATCHED', ascending=False).drop_duplicates('ME')

In [ ]:
pd.merge(phones_matches, )

In [38]:
phone_matches.groupby('MATCHES').count()[['ME']]

,ME
MATCHES,
,236834
DHC,87229
"DHC, DataGov",9888
"DHC, DataGov, Masterfile",7589
"DHC, IQVia",4581
"DHC, IQVia, DataGov",1514
"DHC, IQVia, DataGov, Masterfile",401
"DHC, IQVia, Masterfile",1523
"DHC, Masterfile",27850


In [69]:
DPC[DPC.PHONE_PPD!='None']

,NPI,ME,TOP_CD,PHONE_PPD,PHONE_NPI,IQVIA_ME
1,1588667638,01201861256,20,9042640088,9043881820,0120186125
2,1215930367,86904850022,20,8326784888,2814405006,8690485002
3,1932102084,49558830041,20,4198423000,4198423000,4955883004
6,1578566626,74808810054,20,5208363446,6023446600,7480881005
7,1487657532,03901961441,20,9183426200,4057932229,0390196144
...,...,...,...,...,...,...
1068557,1043898240,00502751123,20,6184533025,9257191473,0050275112
1068844,1790364651,02507862856,20,4233788400,7347306487,0250786285
1069284,1841872231,03547760114,20,5165308095,2034458313,0354776011
1069409,1558945261,00801850448,20,2126624445,6467746912,0080185044


In [74]:
fone_bb = phones_matches.sort_values('MATCHED', ascending=False).drop_duplicates('ME', keep='first')

In [73]:
dpcs = list(DPC.ME)

In [76]:
fone_bb[(fone_bb.ME.isin(dpcs))&(fone_bb.PHONE_PPD=='None')&(fone_bb.MATCHED>0)]

,ME,IQVIA_ME,PHONE_NPI,PHONE_PPD,MATCHED,MATCHES
1373889,04102931390,0410293139,9702074800,None,4,"Symphony, DHC, IQVia, DataGov"
2067684,04813070681,0481307068,4088511000,None,4,"Symphony, DHC, IQVia, DataGov"
729266,69001720279,6900172027,3184283237,None,4,"Symphony, DHC, IQVia, DataGov"
2684404,30501152460,3050115246,4803212000,None,4,"Symphony, DHC, IQVia, DataGov"
2316239,05575830554,0557583055,3047997400,None,4,"Symphony, DHC, IQVia, DataGov"
...,...,...,...,...,...,...
832344,02507802683,0250780268,8017146570,None,1,DHC
1083372,00512771211,0051277121,9166468431,None,1,Symphony
2547876,04202130114,0420213011,4078944693,None,1,DataGov
2547304,01611113150,0161111315,6309862800,None,1,DataGov


In [ ]:
85,676
49,914

In [86]:
new = 85676
x = (671724+new)/818426
print(x)
print(x*0.5653)

0.9254349202004823
0.5231483603893327
